In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from matplotlib import animation
import matplotlib
import time
import sys
import math
from matplotlib import animation, rc, rcParams
from IPython.display import HTML
import time
import csv
from IPython.display import display
from sklearn.model_selection import train_test_split


# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
############## READING CSV ################
# read csv file into position and velocity lists
file_name = '/content/drive/My Drive/Sp2020/CS522/Final Project/Datasets/self_propelled_particles/swarm_03242020_3000steps.csv'
my_data = []
with open(file_name) as csvDataFile:
  csvReader = csv.reader(csvDataFile)
  for row in csvReader:
    my_data.append(row)
pos_from_file = []
vel_from_file =[]
pos_line = []
vel_line = []

for i in range(len(my_data)):
  for j in range(int(len(my_data[0])/2)):
    try:
      pos_line_str = my_data[i][j*2].lstrip('[').rstrip(']').split()
      vel_line_str = my_data[i][j*2+1].lstrip('[').rstrip(']').split()
    except:
      pass
    pos_line.append([float(pos_line_str[0]), float(pos_line_str[1])])
    vel_line.append([float(vel_line_str[0]), float(vel_line_str[1])])
  pos_from_file.append(pos_line)
  vel_from_file.append(vel_line)
  pos_line = []
  vel_line = []

pos_from_file = np.array(pos_from_file)
vel_from_file = np.array(vel_from_file)

In [0]:
def calculate_POD_basis(snapshots):
    '''
    :param snapshots: snapshots imported as 2-D array of size (# of elements, # of snapshots)

    :return:
    '''
    n_time = len(snapshots)
    snapshots = np.transpose(snapshots)
    # singular value decomposition
    U,S,V = np.linalg.svd(snapshots)

    U = U.real
    S = S.real
    V = V.real

    I = np.argsort(S)[::-1]
    S = S[I]
    U = U[:, I]
    
    #pod_basis = create_basis(U, S, energy=0.9)
    

    pod_basis = U[:,1]
    pod_basis = np.reshape(pod_basis, [-1,1])


    _,m = pod_basis.shape
    print('pod basis shape: ', pod_basis.shape, 'm: ',m)
    sing = np.diag(S[0:m])
    tail = np.zeros([m,n_time-m])
    mid = np.concatenate((sing, tail), axis=1)
    
    recon = pod_basis @ mid @ V
    return np.transpose(recon)


def create_basis(V, D, energy):
    '''
    Creates basis functions for POD using SVD decomposition
    :param V: eigenvectors of covariance matrix
    :param D: diagonal matrix with eigenvalues of covariance matrix
    :param energy: parameter for how much "energy" is desired to be capture in POD basis (usually 95%+)
    :return: POD basis phi
    '''


    cum_sum = 0
    total_sum = sum(D).real
    m = 0  # Calculate number of POD bases that will be needed

    for i in range(len(D)):
        cum_sum += D[i].real
        percentage_energy = cum_sum/total_sum
        if percentage_energy < energy:
            m = i
        else:
            m = i
            break

    # Create new matrix based on POD values at each of the sub domains given by value in regionDictionary
    # phi is (n, m) for n = number of elements in mesh, m = number of elements in POD
    phi = V[:, 0:m + 1]
    return phi.real


In [9]:
pos_for_pod = np.reshape(pos_from_file, [len(pos_from_file),-1])
recon = calculate_POD_basis(pos_for_pod)
recon = np.reshape(np.array(recon), [len(pos_from_file),-1,2])


pod basis shape:  (64, 1) m:  1


In [10]:
############ REPLAYING FILES ############
print("Number of time steps: ", len(recon))
anim_lst = recon.copy()
# Animating a single agent, assuming agents live on a 2D plane
# initializing animation parameters
fig = plt.figure()

min_x = np.amin(recon[:,:,0])
max_x = np.amax(recon[:,:,0])
min_y = np.amin(recon[:,:,1])
max_y = np.amax(recon[:,:,1])

ax = plt.axes(xlim=(min_x,max_x),ylim=(min_y,max_y))

anim_lst = np.array(anim_lst)
x = anim_lst[:,:,0]
y = anim_lst[:,:,1]
d, = ax.plot(x[0],y[0], c='0.3',marker='.',linestyle='None')

# embedding animation, problem is that embedding size is limited
anim_lst = anim_lst.tolist()
def animate(i):
  try:
    dots = anim_lst[0][:]
    d.set_data([dot[0] for dot in dots],[dot[1] for dot in dots])
    anim_lst.pop(0)
    return d,
  except:
    "done"
# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=300, interval=50)
# anim.save('training_1000steps.mp4',writer=animation.FFMpegWriter(fps=10))
rc('animation', html='jshtml')
anim


Output hidden; open in https://colab.research.google.com to view.